In [46]:
# @title
import warnings
warnings.filterwarnings('ignore')

try:
    import cirq
except ImportError:
    print('installing cirq...')
    %pip install cirq --quiet
    import cirq
    print('installed cirq.')

import matplotlib.pyplot as plt
import random
import numpy as np

In [47]:
#Process 1: Sending

#Step 1: Alice Generates a Random Key
def generate_classical_key(num_bits): 
    #Step 1: Alice generates a random secret key of classical bits
    key = [random.choice(['0','1']) for _ in range(num_bits)]
    return ''.join (key)

def alice_prepare_bb84(key, num_bits):
    #Create one qubit per bit
    qubits = [cirq.NamedQubit(f'a{i}') for i in range(num_bits)]
    circuit = cirq.Circuit()
    bases = []

    #Convert classical bits to qubit states and apply random bases
    for i, (qubit, bit) in enumerate(zip(qubits,key)):
        basis = random.choice([0,1]) #0: rectilinear(Z), 1:diagonal(X)
        bases.append(basis)

        #Convert classical bit to qubit state
        if bit == '1':
            circuit.append(cirq.X(qubit))

        #Apply H-gate for diagonal basis
        if basis == 1:
            circuit.append(cirq.H(qubit)) 
            #Applies H-gate for bit randomly chosen to be in X-basis
    return circuit, bases, qubits

#Generate classical key
num_bits= 16
secret_key = generate_classical_key(num_bits)
print(f"Alice's Classical Secret Key ({num_bits} bits): {secret_key}")

#Alice prepares quantum states
alice_circuit, alice_bases, qubits = alice_prepare_bb84(secret_key, num_bits)
print("\nAlice's Basis Choices (0=Rectilinear, 1=Diagonal):", alice_bases)

#Print the Circuit
print("\nAlice's Circuit for All Bits:")
print(alice_circuit)

#Print state preparation for each bit
print("\nState Preparation for Each Bit:")
for i, (bit, basis) in enumerate(zip(secret_key, alice_bases)):
    print(f"Bit {i} (key={bit}, basis={basis}): q{i}")


Alice's Classical Secret Key (16 bits): 0110011011010111

Alice's Basis Choices (0=Rectilinear, 1=Diagonal): [1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0]

Alice's Circuit for All Bits:
a0: ────H───────

a1: ────X───────

a2: ────X───────

a5: ────X───────

a6: ────X───H───

a7: ────H───────

a8: ────X───────

a9: ────X───H───

a10: ───H───────

a11: ───X───────

a13: ───X───H───

a14: ───X───H───

a15: ───X───────

State Preparation for Each Bit:
Bit 0 (key=0, basis=1): q0
Bit 1 (key=1, basis=0): q1
Bit 2 (key=1, basis=0): q2
Bit 3 (key=0, basis=0): q3
Bit 4 (key=0, basis=0): q4
Bit 5 (key=1, basis=0): q5
Bit 6 (key=1, basis=1): q6
Bit 7 (key=0, basis=1): q7
Bit 8 (key=1, basis=0): q8
Bit 9 (key=1, basis=1): q9
Bit 10 (key=0, basis=1): q10
Bit 11 (key=1, basis=0): q11
Bit 12 (key=0, basis=0): q12
Bit 13 (key=1, basis=1): q13
Bit 14 (key=1, basis=1): q14
Bit 15 (key=1, basis=0): q15


In [48]:
#Process 2: Intercepting – Hannah

def eve_intercept_bb84(alice_circuit, qubits, num_bits):
    # Create Eve's probe qubits (one per Alice qubit)
    eve_probes = [cirq.NamedQubit(f'e{i}') for i in range(num_bits)]
    eve_bases = [random.choice([0, 1]) for _ in range(num_bits)]  # Eve randomly chooses Z or X basis

    # Add entanglement and measurement operations to Alice's circuit
    for i in range(num_bits):
        qubit_0 = qubits[i]
        eve_probe = eve_probes[i]

        # Step 1: Eve prepares probe in |0⟩ state (default)

        # Step 2: Entangle with Alice's qubit using CNOT (Alice's qubit as control)
        alice_circuit.append(cirq.CNOT(qubit_0, eve_probe))

        # Step 3: Eve measures her probe in a random basis (Z or X)
        if eve_bases[i] == 1:  # X basis
            alice_circuit.append(cirq.H(eve_probe))
        alice_circuit.append(cirq.measure(eve_probe, key=f'eve_m{i}'))

    return alice_circuit, eve_probes, eve_bases

# Eve intercepts and modifies Alice's circuit
alice_circuit, eve_probes, eve_bases = eve_intercept_bb84(alice_circuit, qubits, num_bits)

# Print modified circuit and Eve's bases
print("\nCircuit After Eve's Interception (Process 2):")
print(alice_circuit)
print("\nEve's Basis Choices (0=Rectilinear, 1=Diagonal):", eve_bases)


Circuit After Eve's Interception (Process 2):
        ┌────┐   ┌─────────────────────┐   ┌────────────────┐
a0: ─────H────────────────────@──────────────────────────────────────────────────────────────
                              │
a1: ─────X────────────────────┼@─────────────────────────────────────────────────────────────
                              ││
a2: ─────X────────────────────┼┼@────────────────────────────────────────────────────────────
                              │││
a3: ──────@───────────────────┼┼┼────────────────────────────────────────────────────────────
          │                   │││
a4: ──────┼@──────────────────┼┼┼────────────────────────────────────────────────────────────
          ││                  │││
a5: ─────X┼┼──────────────────┼┼┼@───────────────────────────────────────────────────────────
          ││                  ││││
a6: ─────X┼┼──────H───────────┼┼┼┼──────────────────────@────────────────────────────────────
          ││                  │

In [49]:
#Process 3: Receiving

def bob_measure_bb84(alice_circuit, qubits, num_bits):
    bob_bases = [random.choice([0, 1]) for _ in range(num_bits)]  # 0: Z basis, 1: X basis
    bob_circuit = alice_circuit.copy()  # Copy to avoid modifying original
    bob_measurements = []

    # Append measurement operations
    for i, qubit in enumerate(qubits):
        if bob_bases[i] == 1:  # X basis
            bob_circuit.append(cirq.H(qubit))
        bob_circuit.append(cirq.measure(qubit, key=f'm{i}'))

    # Simulate the entire circuit
    sim = cirq.Simulator()
    result = sim.run(bob_circuit, repetitions=1)

    # Extract measurements, ensuring single-run data
    for i in range(num_bits):
        measurement = result.measurements.get(f'm{i}')
        if measurement is None:
            raise ValueError(f"Measurement key 'm{i}' not found in results")
        # Extract the single measurement value (shape is [1, 1] for repetitions=1)
        bob_measurements.append(int(measurement[0][0]))

    return bob_bases, bob_measurements

# Call Bob's measurement function
bob_bases, bob_measurements = bob_measure_bb84(alice_circuit, qubits, num_bits)

# Print Bob's measurement results
print("\nBob's Measurement Results:")
for i in range(num_bits):
    print(f"Bit {i} (basis={bob_bases[i]}, measurement={bob_measurements[i]}): a{i}")



Bob's Measurement Results:
Bit 0 (basis=1, measurement=0): a0
Bit 1 (basis=0, measurement=1): a1
Bit 2 (basis=0, measurement=1): a2
Bit 3 (basis=1, measurement=0): a3
Bit 4 (basis=0, measurement=0): a4
Bit 5 (basis=1, measurement=0): a5
Bit 6 (basis=0, measurement=1): a6
Bit 7 (basis=1, measurement=0): a7
Bit 8 (basis=1, measurement=1): a8
Bit 9 (basis=1, measurement=1): a9
Bit 10 (basis=0, measurement=0): a10
Bit 11 (basis=1, measurement=1): a11
Bit 12 (basis=0, measurement=0): a12
Bit 13 (basis=1, measurement=0): a13
Bit 14 (basis=1, measurement=1): a14
Bit 15 (basis=0, measurement=1): a15


In [50]:
# Process 4: Comparing (Reconciliation Stage)

def reconcile_keys(alice_bases, bob_bases, secret_key, bob_measurements, num_bits):
    sifted_key_alice = []
    sifted_key_bob = []
    matching_indices = []
    
    # Compare bases and bits for each position
    print("\nBit-by-Bit Comparison (Alice vs. Bob):")
    print("Index | Alice Bit | Alice Basis | Bob Bit | Bob Basis | Bases Match | Bits Match")
    print("-" * 70)
    for i in range(num_bits):
        alice_bit = secret_key[i]
        alice_basis = alice_bases[i]
        bob_bit = str(bob_measurements[i])
        bob_basis = bob_bases[i]
        bases_match = alice_basis == bob_basis
        bits_match = str(alice_bit == bob_bit) if bases_match else "-"
        
        # Print comparison with explicit string formatting
        print(f"{i:5d} | {alice_bit:9s} | {alice_basis:11d} | {bob_bit:7s} | {bob_basis:9d} | {str(bases_match):11s} | {str(bits_match):8s}")
        
        # Form sifted key for matching bases
        if bases_match:
            sifted_key_alice.append(alice_bit)
            sifted_key_bob.append(bob_bit)
            matching_indices.append(i)
    
    # Error checking: Calculate error rate
    errors = sum(1 for a, b in zip(sifted_key_alice, sifted_key_bob) if a != b)
    error_rate = errors / len(sifted_key_alice) if sifted_key_alice else 0
    return sifted_key_alice, sifted_key_bob, matching_indices, error_rate

sifted_key_alice, sifted_key_bob, matching_indices, error_rate = reconcile_keys(
    alice_bases, bob_bases, secret_key, bob_measurements, num_bits
)
print(f"\nSifted Key (Alice): {''.join(sifted_key_alice)}")
print(f"Sifted Key (Bob): {''.join(sifted_key_bob)}")
print(f"Matching Indices: {matching_indices}")
print(f"Error Rate: {error_rate:.3f}")


Bit-by-Bit Comparison (Alice vs. Bob):
Index | Alice Bit | Alice Basis | Bob Bit | Bob Basis | Bases Match | Bits Match
----------------------------------------------------------------------
    0 | 0         |           1 | 0       |         1 | True        | True    
    1 | 1         |           0 | 1       |         0 | True        | True    
    2 | 1         |           0 | 1       |         0 | True        | True    
    3 | 0         |           0 | 0       |         1 | False       | -       
    4 | 0         |           0 | 0       |         0 | True        | True    
    5 | 1         |           0 | 0       |         1 | False       | -       
    6 | 1         |           1 | 1       |         0 | False       | -       
    7 | 0         |           1 | 0       |         1 | True        | True    
    8 | 1         |           0 | 1       |         1 | False       | -       
    9 | 1         |           1 | 1       |         1 | True        | True    
   10 | 0         

In [51]:
# Process 5: Eve’s Measurements
from collections import Counter

def povm_measurement(probabilities):
    """Simulate POVM outcome based on normalized probability list."""
    return np.random.choice(len(probabilities), p=probabilities)

def eve_measure_povm(eve_probes, matching_indices, alice_circuit, alice_bases):
    povm_results = []
    outcome_probs_all = []

    # Define simple POVM effects (E0, E1, E_ambiguous) as diagonal operators
    E0 = np.array([[0.9, 0.0], [0.0, 0.1]])         # Suggests bit = 0
    E1 = np.array([[0.1, 0.0], [0.0, 0.9]])         # Suggests bit = 1
    E_ambiguous = np.array([[0.5, 0.0], [0.0, 0.5]])  # Inconclusive
    povm_elements = [E0, E1, E_ambiguous]

    sim = cirq.Simulator()

    for i in matching_indices:
        probe = eve_probes[i]
        basis = alice_bases[i]

        # Build circuit for Eve's probe up to this point
        eve_circuit = cirq.Circuit()

        for moment in alice_circuit:
            for op in moment:
                if probe in op.qubits:
                    eve_circuit.append(op)

        # Align Eve's measurement basis with Alice’s
        if basis == 1:
            eve_circuit.append(cirq.H(probe))

        # Simulate to get final state vector
        result = sim.simulate(eve_circuit)
        state_vector = result.final_state_vector
        index = result.qubit_map[probe]

        # Extract reduced density matrix for Eve’s qubit
        eve_dm = np.zeros((2, 2), dtype=complex)
        for b in range(len(state_vector)):
            if ((b >> index) & 1) == 0:
                eve_dm[0, 0] += np.abs(state_vector[b]) ** 2
            else:
                eve_dm[1, 1] += np.abs(state_vector[b]) ** 2

        # Compute probabilities from POVM elements
        outcome_probs = [np.real(np.trace(np.dot(E, eve_dm))) for E in povm_elements]
        normalized_probs = np.array(outcome_probs) / np.sum(outcome_probs)

        outcome_probs_all.append(normalized_probs.tolist())
        measured = povm_measurement(normalized_probs)
        povm_results.append(measured)

    return povm_results, outcome_probs_all

def renyi_information(povm_results, sifted_key_alice):
    """Compute Rényi information (order 2) between Eve’s POVM outcomes and Alice’s bits."""
    joint_counts = Counter()
    eve_counts = Counter()

    for e_outcome, a_bit in zip(povm_results, sifted_key_alice):
        joint_counts[(e_outcome, a_bit)] += 1
        eve_counts[e_outcome] += 1

    total = len(sifted_key_alice)
    renyi_sum = 0

    for (e, a), joint in joint_counts.items():
        p_ea = joint / total
        p_e = eve_counts[e] / total
        renyi_sum += (p_ea ** 2) / p_e

    return np.log2(renyi_sum) if renyi_sum > 0 else 0.0

# --- Run Eve's POVM Measurement ---
povm_results, povm_probs = eve_measure_povm(
    eve_probes, matching_indices, alice_circuit, alice_bases
)

# --- Display POVM Probabilities ---
print("\nEve's POVM Outcome Probabilities (per matching index):")
for idx, probs in zip(matching_indices, povm_probs):
    print(f"Bit {idx}: P(0)={probs[0]:.3f}, P(1)={probs[1]:.3f}, P(?)={probs[2]:.3f}")

# --- Display Eve's Outcomes ---
print("\nEve's POVM Outcomes (0=bit 0, 1=bit 1, 2=ambiguous):")
print(povm_results)

# --- Compute Rényi Information ---
I_renyi = renyi_information(povm_results, sifted_key_alice)
print(f"\nRényi Information (Order 2): {I_renyi:.3f} bits")



Eve's POVM Outcome Probabilities (per matching index):
Bit 0: P(0)=0.600, P(1)=0.067, P(?)=0.333
Bit 1: P(0)=0.600, P(1)=0.067, P(?)=0.333
Bit 2: P(0)=0.600, P(1)=0.067, P(?)=0.333
Bit 4: P(0)=0.600, P(1)=0.067, P(?)=0.333
Bit 7: P(0)=0.600, P(1)=0.067, P(?)=0.333
Bit 9: P(0)=0.600, P(1)=0.067, P(?)=0.333
Bit 12: P(0)=0.600, P(1)=0.067, P(?)=0.333
Bit 13: P(0)=0.600, P(1)=0.067, P(?)=0.333
Bit 14: P(0)=0.600, P(1)=0.067, P(?)=0.333
Bit 15: P(0)=0.600, P(1)=0.067, P(?)=0.333

Eve's POVM Outcomes (0=bit 0, 1=bit 1, 2=ambiguous):
[0, 2, 2, 0, 0, 2, 2, 2, 0, 0]

Rényi Information (Order 2): -0.737 bits


In [52]:
#Process 6: Privacy Amplification
def privacy_amplification(sifted_key_alice, sifted_key_bob, matching_indices, final_key_length):
    # Select a random subset of sifted key bits for the final key
    if len(sifted_key_alice) < final_key_length:
        final_key_length = len(sifted_key_alice)
    
    indices = random.sample(range(len(sifted_key_alice)), final_key_length)
    amplified_key_alice = [sifted_key_alice[i] for i in indices]
    amplified_key_bob = [sifted_key_bob[i] for i in indices]
    amplified_indices = [matching_indices[i] for i in indices]
    
    return amplified_key_alice, amplified_key_bob, amplified_indices

# Call privacy amplification
final_key_length = max(1, len(sifted_key_alice) // 2)  # Half the sifted key length
amplified_key_alice, amplified_key_bob, amplified_indices = privacy_amplification(
    sifted_key_alice, sifted_key_bob, matching_indices, final_key_length
)
print("\nAmplified Key (Alice):", ''.join(amplified_key_alice))
print("Amplified Key (Bob):", ''.join(amplified_key_bob))
print("Amplified Indices:", amplified_indices)

# Fix output bug in state preparation (for clarity, not modifying original code)
print("\nState Preparation for Each Bit (Corrected Qubit Names):")
for i, (bit, basis, qubit) in enumerate(zip(secret_key, alice_bases, qubits)):
    print(f"Bit {i} (key={bit}, basis={basis}, qubit={qubit})")


Amplified Key (Alice): 10011
Amplified Key (Bob): 10001
Amplified Indices: [1, 0, 12, 13, 9]

State Preparation for Each Bit (Corrected Qubit Names):
Bit 0 (key=0, basis=1, qubit=a0)
Bit 1 (key=1, basis=0, qubit=a1)
Bit 2 (key=1, basis=0, qubit=a2)
Bit 3 (key=0, basis=0, qubit=a3)
Bit 4 (key=0, basis=0, qubit=a4)
Bit 5 (key=1, basis=0, qubit=a5)
Bit 6 (key=1, basis=1, qubit=a6)
Bit 7 (key=0, basis=1, qubit=a7)
Bit 8 (key=1, basis=0, qubit=a8)
Bit 9 (key=1, basis=1, qubit=a9)
Bit 10 (key=0, basis=1, qubit=a10)
Bit 11 (key=1, basis=0, qubit=a11)
Bit 12 (key=0, basis=0, qubit=a12)
Bit 13 (key=1, basis=1, qubit=a13)
Bit 14 (key=1, basis=1, qubit=a14)
Bit 15 (key=1, basis=0, qubit=a15)
